In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
base_url = "https://www.cricbuzz.com"
page_url = "https://www.cricbuzz.com/cricket-series/3472/indian-premier-league-2021/matches"
driver.get(page_url)
time.sleep(3)  # Let JS load
page_source = driver.page_source
driver.quit()
soup = BeautifulSoup(page_source, "html.parser")
match_data = []
matches = soup.select("div.cb-col-100.cb-col.cb-series-matches")
year = "2021"
for match in matches:
    date = match.select_one('span.ng-binding').text.strip()
    match_date = date.split(",")[0].strip()
    match_item = match.select_one("a.text-hvr-underline")
    if not match_item:
        continue  
    match_detail = match_item.select_one("span").text.strip().split(',')
    match_name = match_detail[0]   
    match_no = str(len(match_data)+1)
    match_href = base_url + match_item['href']
    playoffs = ["Qualifier 1", "Qualifier 2", "Eliminator", "Final"]
    is_playoff = match_detail[1].strip() in playoffs
    match_type = "Playoff" if is_playoff else "League"
    match_time_tag = match.select_one('span.schedule-date')
    match_time = match_time_tag.text.strip() if match_time_tag else ""
    match_venue_tag = match.select_one("div.text-gray")
    match_venue = match_venue_tag.text.strip() if match_venue_tag else ""
    match_teams = match_name.split(" vs ")
    if len(match_teams) != 2:
        continue
    match_team1 = match_teams[0].strip()
    match_team2 = match_teams[1].strip()
    result_tag = match.select_one("a.cb-text-complete")
    result = result_tag.text.strip() if result_tag else ""
    winning_team = result.split("won")[0].strip() if "won" in result else ""
    match_data.append({
        "match_no": match_no,
        "match_name": match_name,
        "match_type": match_type,
        "match_date": match_date + ", " + year,
        "match_time": match_time,
        "match_venue": match_venue,
        "match_href": match_href,
        "match_team1": match_team1,
        "match_team2": match_team2,
        "winning_team": winning_team.upper()
    })
df = pd.DataFrame(match_data)
df.to_csv("ipl 2021 data.csv", index=False)
print("Scraping completed and data saved.")

Scraping completed and data saved.
